In [1]:
# Example 1: Load Flow Admittance Matrix
# Connect to PowerFactory, extract network, build Y-matrix, run load flow
import sys

PF_PYTHON_API_PATH = r"C:\Program Files\DIgSILENT\PowerFactory 2024 SP4A\Python\3.12" # Path to PowerFactory Python API
sys.path.insert(0, PF_PYTHON_API_PATH)
import powerfactory as pf

# Connect to PowerFactory
app = pf.GetApplicationExt()

# Import PFD file and initialize project
from admittance_matrix.utils import import_pfd_file

pfd_file_project = "grid_models\\Radial System.pfd" # File path to the PFD file
pfd_project_name = "Radial System" # IntPrj name
import_pfd_file(app, pfd_file_project, pfd_project_name)  # Change to your project
app.Show()

In [ ]:
from admittance_matrix import Network

# Create network and build matrices
# Use simplify_topology=True to merge buses connected by closed switches
net = Network(app, base_mva=100.0, simplify_topology=False)

In [13]:
import pandas as pd

# View load flow Y-matrix as DataFrame
Y_load_flow = net.Y_lf
pd.DataFrame(Y_load_flow, index=net.bus_names, columns=net.bus_names).head().round(2)

,Bus 1,Bus 10,Bus 11,Bus 12,Bus 13,Bus 14,Bus 15,Bus 16,Bus 17,Bus 18,...,Bus 20,Bus 21,Bus 3,Bus 4,Bus 5,Bus 6,Bus 7,Bus 8,Bus 9,Bus X
Bus 1,0.00-9.73j,0.00+ 0.00j,0.000000e+00+0.000000e+ 00j,0.00+ 0.00j,0.00+ 0.00j,0.0+ 0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,...,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+ 0.0j,0.000000e+00+0.000000e+ 00j
Bus 10,0.00+0.00j,0.00-19.47j,-0.000000e+00+1.000000e+ 01j,0.00+ 0.00j,0.00+ 0.00j,0.0+ 0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,...,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,-0.0+10.0j,0.000000e+00+0.000000e+ 00j
Bus 11,0.00+0.00j,-0.00+10.00j,1.600000e+09-1.947000e+ 01j,-0.00+10.00j,0.00+ 0.00j,0.0+ 0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,...,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+ 0.0j,-1.600000e+09+0.000000e+ 00j
Bus 12,0.00+0.00j,0.00+ 0.00j,-0.000000e+00+1.000000e+ 01j,0.00-19.47j,-0.00+10.00j,0.0+ 0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,...,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+ 0.0j,0.000000e+00+0.000000e+ 00j
Bus 13,0.00+0.00j,0.00+ 0.00j,0.000000e+00+0.000000e+ 00j,-0.00+10.00j,0.00-19.47j,-0.0+10.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,...,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+ 0.0j,0.000000e+00+0.000000e+ 00j


In [4]:
# Run load flow
success = net.run_load_flow()
print(f"Load flow converged: {success}")
print(f"Generators found: {len(net.gen_data)}")

21
Number of generators extracted: 21
Number of voltage sources extracted: 0
Number of external grids extracted: 0
Load flow converged: True
Generators found: 21


In [5]:
# View load flow results
lf_data = [{"Bus": k, "V (p.u.)": v.voltage_pu, "Angle (°)": v.angle_deg} for k, v in net.lf_results.items()]
pd.DataFrame(lf_data)

,Bus,V (p.u.),Angle (°)
0,Bus 1,1.0,0.000000e+00
1,Bus 10,1.0,-9.642883e-08
2,Bus 11,1.0,-1.039919e-07
3,Bus 12,1.0,-1.102944e-07
4,Bus 13,1.0,-1.159667e-07
5,Bus 14,1.0,-1.210087e-07
6,Bus 15,1.0,-1.254205e-07
7,Bus 16,1.0,-1.292020e-07
8,Bus 17,1.0,-1.323533e-07
9,Bus 18,1.0,-1.348743e-07
